In [3]:
import pandas as pd
excel_with_path = '/eos/user/f/franzoni/SWAN_projects/proton/online_retail_II.xlsx'
df1 = pd.read_excel (excel_with_path,'Year 2009-2010')
#print (df1)

       Invoice StockCode                          Description  Quantity  \
0       489434     85048  15CM CHRISTMAS GLASS BALL 20 LIGHTS        12   
1       489434    79323P                   PINK CHERRY LIGHTS        12   
2       489434    79323W                  WHITE CHERRY LIGHTS        12   
3       489434     22041         RECORD FRAME 7" SINGLE SIZE         48   
4       489434     21232       STRAWBERRY CERAMIC TRINKET BOX        24   
5       489434     22064           PINK DOUGHNUT TRINKET POT         24   
6       489434     21871                  SAVE THE PLANET MUG        24   
7       489434     21523   FANCY FONT HOME SWEET HOME DOORMAT        10   
8       489435     22350                            CAT BOWL         12   
9       489435     22349       DOG BOWL , CHASING BALL DESIGN        12   
10      489435     22195         HEART MEASURING SPOONS LARGE        24   
11      489435     22353   LUNCHBOX WITH CUTLERY FAIRY CAKES         12   
12      489436    48173C 

In [4]:
df2 = pd.read_excel (excel_with_path,'Year 2010-2011')
#print (df2)

       Invoice StockCode                          Description  Quantity  \
0       536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1       536365     71053                  WHITE METAL LANTERN         6   
2       536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3       536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4       536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   
5       536365     22752         SET 7 BABUSHKA NESTING BOXES         2   
6       536365     21730    GLASS STAR FROSTED T-LIGHT HOLDER         6   
7       536366     22633               HAND WARMER UNION JACK         6   
8       536366     22632            HAND WARMER RED POLKA DOT         6   
9       536368     22960             JAM MAKING SET WITH JARS         6   
10      536368     22913          RED COAT RACK PARIS FASHION         3   
11      536368     22912       YELLOW COAT RACK PARIS FASHION         3   
12      536368     22914 

In [6]:
df = pd.concat([df1, df2])
print (df)

       Invoice StockCode                          Description  Quantity  \
0       489434     85048  15CM CHRISTMAS GLASS BALL 20 LIGHTS        12   
1       489434    79323P                   PINK CHERRY LIGHTS        12   
2       489434    79323W                  WHITE CHERRY LIGHTS        12   
3       489434     22041         RECORD FRAME 7" SINGLE SIZE         48   
4       489434     21232       STRAWBERRY CERAMIC TRINKET BOX        24   
5       489434     22064           PINK DOUGHNUT TRINKET POT         24   
6       489434     21871                  SAVE THE PLANET MUG        24   
7       489434     21523   FANCY FONT HOME SWEET HOME DOORMAT        10   
8       489435     22350                            CAT BOWL         12   
9       489435     22349       DOG BOWL , CHASING BALL DESIGN        12   
10      489435     22195         HEART MEASURING SPOONS LARGE        24   
11      489435     22353   LUNCHBOX WITH CUTLERY FAIRY CAKES         12   
12      489436    48173C 

In [7]:
pickle_with_path = '/eos/user/f/franzoni/SWAN_projects/proton/online_retail_II.pkl'
df.to_pickle(pickle_with_path)

In [8]:
df_ready = pd.read_pickle(pickle_with_path)
print(df_ready)

       Invoice StockCode                          Description  Quantity  \
0       489434     85048  15CM CHRISTMAS GLASS BALL 20 LIGHTS        12   
1       489434    79323P                   PINK CHERRY LIGHTS        12   
2       489434    79323W                  WHITE CHERRY LIGHTS        12   
3       489434     22041         RECORD FRAME 7" SINGLE SIZE         48   
4       489434     21232       STRAWBERRY CERAMIC TRINKET BOX        24   
5       489434     22064           PINK DOUGHNUT TRINKET POT         24   
6       489434     21871                  SAVE THE PLANET MUG        24   
7       489434     21523   FANCY FONT HOME SWEET HOME DOORMAT        10   
8       489435     22350                            CAT BOWL         12   
9       489435     22349       DOG BOWL , CHASING BALL DESIGN        12   
10      489435     22195         HEART MEASURING SPOONS LARGE        24   
11      489435     22353   LUNCHBOX WITH CUTLERY FAIRY CAKES         12   
12      489436    48173C 

In [10]:
import os

# GF TODO: check if excel present, otherwise download it
# wget http://archive.ics.uci.edu/ml/machine-learning-databases/00502/online_retail_II.xlsx
pickle_with_path = '/eos/user/f/franzoni/SWAN_projects/proton/online_retail_II.pkl'
excel_with_path  = '/eos/user/f/franzoni/SWAN_projects/proton/online_retail_II.xlsx'

df = None

if os.path.isfile(pickle_with_path):
    print('-> picke already exists, much faster\n')
    df = pd.read_pickle(pickle_with_path)
    print(df)
else:
    print('-> picke does not exist, go to excel, and creat it\n')
    df1 = pd.read_excel (excel_with_path,'Year 2009-2010')
    df2 = pd.read_excel (excel_with_path,'Year 2010-2011')
    df = pd.concat([df1, df2])
    df.to_pickle(pickle_with_path)
    print (df)

picke does not exist, go to excel, and creat it
       Invoice StockCode                          Description  Quantity  \
0       489434     85048  15CM CHRISTMAS GLASS BALL 20 LIGHTS        12   
1       489434    79323P                   PINK CHERRY LIGHTS        12   
2       489434    79323W                  WHITE CHERRY LIGHTS        12   
3       489434     22041         RECORD FRAME 7" SINGLE SIZE         48   
4       489434     21232       STRAWBERRY CERAMIC TRINKET BOX        24   
5       489434     22064           PINK DOUGHNUT TRINKET POT         24   
6       489434     21871                  SAVE THE PLANET MUG        24   
7       489434     21523   FANCY FONT HOME SWEET HOME DOORMAT        10   
8       489435     22350                            CAT BOWL         12   
9       489435     22349       DOG BOWL , CHASING BALL DESIGN        12   
10      489435     22195         HEART MEASURING SPOONS LARGE        24   
11      489435     22353   LUNCHBOX WITH CUTLERY FAI